In [1]:
import snowflake.connector
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [4]:
username=os.getenv(key='snowflake_username')
password=os.getenv(key='SNOWSQL_PWD')
accountName=os.getenv(key='account')

In [5]:
conn=snowflake.connector.connect(
    user=username,
    password=password,
    account=accountName
)

In [6]:
conn.cursor().execute("""
    ALTER SESSION SET QUERY_TAG='EndOfMonthFinancials'
""")

<h3>Create a Warehouse</h3>

In [7]:
conn.cursor().execute("""CREATE WAREHOUSE IF NOT EXISTS 
                            third_warehouse
                        WITH 
                            WAREHOUSE_SIZE=XSMALL
                            MIN_CLUSTER_COUNT=1
                            MAX_CLUSTER_COUNT=3
                            SCALING_POLICY="Economy"  -- Default is "Standard"
                            INITIALLY_SUSPENDED=TRUE
                            AUTO_SUSPEND=50
                            AUTO_RESUME=TRUE
                            COMMENT="This is our third warehouse";
    """)

In [8]:
conn.cursor().execute("""
    USE WAREHOUSE
        third_warehouse
""")

<h3>Create a Database</h3>

In [9]:
conn.cursor().execute("""
    CREATE DATABASE IF NOT EXISTS
        python_test_db;
""")

In [10]:
conn.cursor().execute("""
    USE DATABASE
        python_test_db;
""")

<h3>Create a Schema</h3>

In [12]:
conn.cursor().execute("""
    CREATE SCHEMA IF NOT EXISTS
        python_test_db.test_schema
""")

In [13]:
conn.cursor().execute("""
        USE SCHEMA
            python_test_db.test_schema
""")

<h3>Create a Table</h3>

In [15]:
conn.cursor().execute(
    """
        CREATE OR REPLACE TABLE
            python_test_db.test_schema.python_test_table (
                col1 INTEGER,
                col2 STRING
            )
    """
)

<h3>Adding Data to the table</h3>

In [17]:
conn.cursor().execute(
    """
        INSERT INTO 
            python_test_db.test_schema.python_test_table(
                col1,col2
                )
        VALUES
            (
                123,
                'test string1'
            ),
            (
                456,
                'test string2'
            );
    """
)

<h3>Query Data</h3>

In [19]:
col1,col2=conn.cursor().execute(
    """
        SELECT 
            col1,
            col2
        FROM 
            python_test_db.test_schema.python_test_table
    """
).fetchone()

In [20]:
col1,col2

(123, 'test string1')

In [21]:
for col1,col2 in conn.cursor().execute(
    """
        SELECT 
            col1,
            col2
        FROM 
            python_test_db.test_schema.python_test_table
    """
):
    print(f'{col1},{col2}')

123,test string1
456,test string2


<h3>Close the connection</h3>

In [23]:
conn.close()